! This file has problems running on laptop, run on workstation instead

In [ ]:
import xarray as xr
import pyproj
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import cmocean as cmo
import glob
from mpl_toolkits.axes_grid1 import make_axes_locatable
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import numpy as np
import copy
import sys

sys.path.append('../src')

from alldata import AllData

plt.style.use('../src/style_paper')

Input parameters

In [ ]:
#Percentiles for uncertainty ranges
perc = [17,83]

Read annual time series

In [ ]:
ds = xr.open_dataset(f'../data/temperature_ann_ctrl.nc').isel(basin=slice(0,5))
basin = ds.basin
ttime = ds.time+100 #Shift to append control after spinup
tbas = ds.temp
ds.close()

ds = xr.open_dataset(f'../data/temperature_ann_spin.nc').isel(basin=slice(0,5))
ttime2 = ds.time
tbas2 = ds.temp
ds.close()

ds = xr.open_dataset(f'../data/basinmask.nc')
lon = ds.lon
lat = ds.lat
mask = ds.mask
ds.close()

ad = AllData()

Read reanalysis data

In [ ]:
ds = xr.open_dataset('../data/eveline/thetao_sector_timeseries_global-reanalysis-phy-001-026-grepv1_1993_2018.nc')
ds = ds.drop_sel(product=['mean','std'])
evbas = ['eais','ross','amun','wedd','apen']

rean = np.zeros(len(evbas))
r_lo = np.zeros(len(evbas))
r_hi = np.zeros(len(evbas))

for b,bas in enumerate(evbas):
    var = ds[bas].values.flatten()
    rean[b] = np.mean(var)
    r_lo[b] = np.percentile(var,perc[0])
    r_hi[b] = np.percentile(var,perc[1])
    
ds.close()

print(rean)

Read CMIP6 piControl

In [ ]:
ds = xr.open_dataset('../data/temperature_cmip6_piControl.nc')
ds = ds.isel(time=slice(0,-50))
ctime = ds.time + 550
ctbas = ds.temp
ds.close()

Prepare figure

In [ ]:
mpl.rc('lines',linewidth=.5)
mpl.rc('figure.subplot',wspace=.2,right=.9)

btitle = ['a) East Antarctica','b) Ross','d) Amundsen','e) Weddell','f) Peninsula']

nax = [1,2,4,5,6]
alpha = .3

savename = f'../draftfigs/ts_control.png'

Make and save figure

In [ ]:
fig = plt.figure()

#Create map of ocean basins
ax = fig.add_subplot(233,projection=ccrs.SouthPolarStereo())
ax.set_extent([-180, 180, -80, -63], crs=ccrs.PlateCarree())
for b,bas in enumerate(basin.values):
    im = ax.scatter(lon,lat,15*np.nansum(mask[b,:,:,:],axis=0),c=ad.bcol[bas],transform=ccrs.PlateCarree(),label=bas,lw=0,alpha=1)
ax.coastlines(lw=.1)
ax.set_title('c) Ocean basins',loc='left')
gl = ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=False,color='k',linewidth=.1)
gl.xlocator = mticker.FixedLocator(np.arange(-180,210,30))
gl.xformatter = LONGITUDE_FORMATTER

#Plot time series of pi control, spinup and new control per basin
for b,bas in enumerate(basin.values):
    ax = fig.add_subplot(2,3,nax[b])
    temp = tbas.sel(basin=bas)

    ax.plot(ttime,temp,c=ad.bcol[bas],zorder=9)
    ax.plot(ttime2,tbas2.sel(basin=bas),c=ad.bcol[bas],zorder=9)
    ax.plot(ctime,ctbas.sel(basin=bas),color='k')        
        
    ax.fill_between([ctime[0],ttime[-1]],[r_lo[b],r_lo[b]],[r_hi[b],r_hi[b]],color='tab:olive',alpha=alpha,clip_on=False,label=f'Reanalysis',lw=0)
    ax.fill_between([ctime[0],ctime[-1]],[np.percentile(ctbas.sel(basin=bas),perc[0]),np.percentile(ctbas.sel(basin=bas),perc[0])],[np.percentile(ctbas.sel(basin=bas),perc[1]),np.percentile(ctbas.sel(basin=bas),perc[1])],color='k',alpha=alpha,lw=0,clip_on=False)
    ax.fill_between([ttime[-150],ttime[-1]],[np.percentile(temp[-150:],perc[0]),np.percentile(temp[-150:],perc[0])],[np.percentile(temp[-150:],perc[1]),np.percentile(temp[-150:],perc[1])],color=ad.bcol[bas],alpha=alpha,lw=0,clip_on=False)
    
    #Include labels and titles
    ax.set_title(btitle[b],loc='left')

    if nax[b] == 1:
        ax.legend(loc='upper left')
    
    if nax[b] == 5:
        ax.set_xlabel('Time [years]')
    if nax[b] in [1,2]:
        ax.set_xticklabels([])
    if nax[b] not in [1,4]:
        ax.set_yticklabels([])
    if nax[b] == 4:
        ax.text(ctime[0]/2,-1.7,'CMIP 6\npiControl',ha='center')
        
        ax.text(50,-1.7,'spin\nup',ha='center',c='tab:red')
        ax.text(175,-1.7,'CTRL',ha='center',c='tab:red')

    #Some other specifications
    ax.set_ylim([-2,3])
    ax.set_xticks([-450,0,100,250])
    ax.set_xlim([-450,250])
    ax.grid(True,axis='x',linewidth=.3,color='k')
    
fig.supylabel('Subsurface ocean temperature [degC]',fontsize=9)

plt.savefig(savename)
plt.show()